In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from timm import create_model
from tqdm import tqdm

# Configuração do dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Configurações gerais
numero_de_epocas = 20
bs = 8
image_size = 224  # Ajustado para 224x224
dataset = './data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')
numero_de_classes = len(os.listdir(pasta_validacao))
checkpoint_dir = './checkpoints'

# Criar diretório de checkpoints se não existir
os.makedirs(checkpoint_dir, exist_ok=True)

# Transformações para as imagens

transformacoes_de_imagens = {
    'treino': transforms.Compose([
        transforms.Resize(size=[image_size, image_size]),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
    ]),

    'validacao': transforms.Compose([
        transforms.Resize(size=[image_size, image_size]),
        transforms.ToTensor(),
    ])
}

# Carregar datasets
data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

# Mapear os índices com os nomes das classes
indice_para_classe = {v: k for k, v in data['treino'].class_to_idx.items()}
print(f"Mapeamento de índices para classes: {indice_para_classe}")

# Criar DataLoaders
data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True, num_workers=4)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=False, num_workers=4)


# Carregar o modelo ViT pré-treinado
vit_model = create_model('vit_base_patch16_224', pretrained=True)

# Ajustar a última camada para o número de classes
num_features_vit = vit_model.head.in_features
vit_model.head = nn.Sequential(
    nn.Linear(num_features_vit, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, numero_de_classes),
    nn.LogSoftmax(dim=1)
)

# Enviar o modelo para o dispositivo
vit_model = vit_model.to(device)

# Definir a função de erro e o otimizador
funcao_erro = nn.NLLLoss()
otimizador = optim.Adam(vit_model.parameters(), lr=0.0001)

# Função para salvar checkpoints
def salvar_checkpoint(modelo, otimizador, epoca, melhor_acuracia, caminho):
    checkpoint = {
        'epoca': epoca,
        'modelo_state_dict': modelo.state_dict(),
        'otimizador_state_dict': otimizador.state_dict(),
        'melhor_acuracia': melhor_acuracia
    }
    torch.save(checkpoint, caminho)
    print(f"Checkpoint salvo: {caminho}")

# Função para carregar checkpoints
def carregar_checkpoint(caminho, modelo, otimizador):
    checkpoint = torch.load(caminho)
    modelo.load_state_dict(checkpoint['modelo_state_dict'])
    otimizador.load_state_dict(checkpoint['otimizador_state_dict'])
    epoca = checkpoint['epoca']
    melhor_acuracia = checkpoint['melhor_acuracia']
    print(f"Checkpoint carregado: {caminho}, Época: {epoca}, Melhor Acurácia: {melhor_acuracia:.4f}")
    return modelo, otimizador, epoca, melhor_acuracia

# Função para treinamento e validação
def treinar_e_validar(modelo, metrica_erro, otimizador_sgd, epocas=25, iniciar_epoca=0, melhor_acuracia=0.0):
    scaler = torch.cuda.amp.GradScaler()  # Inicializar o GradScaler
    historico = []

    for epoca in range(iniciar_epoca, epocas):
        inicio_epoca = time.time()
        print(f"\nÉpoca {epoca + 1}/{epocas}")

        modelo.train()
        erro_treino = 0.0
        acuracia_treino = 0.0

        for entradas, labels in tqdm(data_loader_treino, desc="Treinando"):
            entradas, labels = entradas.to(device), labels.to(device)
            otimizador_sgd.zero_grad()

            with torch.cuda.amp.autocast():  # Ativar precisão mista
                saidas = modelo(entradas)
                erro = metrica_erro(saidas, labels)

            scaler.scale(erro).backward()
            scaler.step(otimizador_sgd)
            scaler.update()

            erro_treino += erro.item() * entradas.size(0)
            _, preds = torch.max(saidas, 1)
            acuracia_treino += torch.sum(preds == labels.data)

        modelo.eval()
        erro_validacao = 0.0
        acuracia_validacao = 0.0

        with torch.no_grad():
            for entradas, labels in tqdm(data_loader_validacao, desc="Validando"):
                entradas, labels = entradas.to(device), labels.to(device)
                with torch.cuda.amp.autocast():  # Ativar precisão mista
                    saidas = modelo(entradas)
                    erro = metrica_erro(saidas, labels)

                erro_validacao += erro.item() * entradas.size(0)
                _, preds = torch.max(saidas, 1)
                acuracia_validacao += torch.sum(preds == labels.data)

        # Cálculo das métricas
        erro_medio_treino = erro_treino / len(data['treino'])
        acuracia_medio_treino = acuracia_treino.double() / len(data['treino'])
        erro_medio_validacao = erro_validacao / len(data['validacao'])
        acuracia_medio_validacao = acuracia_validacao.double() / len(data['validacao'])

        historico.append([erro_medio_treino, erro_medio_validacao, acuracia_medio_treino, acuracia_medio_validacao])

        print(f"Treino - Erro: {erro_medio_treino:.4f}, Acurácia: {acuracia_medio_treino:.4f}")
        print(f"Validação - Erro: {erro_medio_validacao:.4f}, Acurácia: {acuracia_medio_validacao:.4f}")

        # Salvar checkpoint a cada época
        checkpoint_path = os.path.join(checkpoint_dir, f'Vit_checkpoint_epoca_{epoca + 1}.pth')
        salvar_checkpoint(modelo, otimizador_sgd, epoca + 1, melhor_acuracia, checkpoint_path)

        # Atualizar o melhor modelo
        if acuracia_medio_validacao > melhor_acuracia:
            melhor_acuracia = acuracia_medio_validacao
            torch.save(modelo.state_dict(), 'melhor_modelo_vit.pth')
            print("Melhor modelo salvo!")

    return historico

# Carregar checkpoint (opcional)
checkpoint_path = './checkpoints/checkpoint_epoca_10.pth'  # Altere para o caminho do checkpoint
if os.path.exists(checkpoint_path):
    vit_model, otimizador, iniciar_epoca, melhor_acuracia = carregar_checkpoint(checkpoint_path, vit_model, otimizador)
else:
    iniciar_epoca, melhor_acuracia = 0, 0.0

# Treinar o modelo
# historico = treinar_e_validar(vit_model, funcao_erro, otimizador, numero_de_epocas, iniciar_epoca, melhor_acuracia)


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Usando o dispositivo: cuda:0
Mapeamento de índices para classes: {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


In [2]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# Configuração do dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Configurações gerais
numero_de_epocas = 20
bs = 8
image_size = (96, 72)  # Atualizado para corresponder ao modelo customizado
patches = (16, 16)  # Tamanho do patch do ViT
num_classes = 7  # Atualize de acordo com seu dataset
checkpoint_dir = './checkpoints'

# Criar diretório de checkpoints se não existir
os.makedirs(checkpoint_dir, exist_ok=True)

# Transformações para as imagens
transformacoes_de_imagens = {
    'treino': transforms.Compose([
        transforms.Resize(size=image_size),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
    ]),

    'validacao': transforms.Compose([
        transforms.Resize(size=image_size),
        transforms.ToTensor(),
    ])
}

# Carregar datasets
dataset = './data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')

data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

# Criar DataLoaders
data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True, num_workers=4)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=False, num_workers=4)

# Definir o modelo ViT customizado
from vit import ViT 

vit_model = ViT(
    d_model=256,  # Dimensão do modelo
    num_blks=8,  # Número de blocos do transformer
    nhead=8,  # Número de cabeças de atenção
    patches=patches,
    img_size=image_size,
    first_channel=3,  # Número de canais de entrada
    dropout=0.1,
    report_params_count=True
)

# Classificador para ajustar a saída do ViT
classifier = nn.Sequential(
    nn.Linear(256, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes),
    nn.LogSoftmax(dim=1)  # LogSoftmax para compatibilidade com NLLLoss
)

vit_model.to(device)
classifier.to(device)

# Definir a função de erro e o otimizador
funcao_erro = nn.NLLLoss()  # Negative Log Likelihood Loss
otimizador = optim.Adam(
    list(vit_model.parameters()) + list(classifier.parameters()), 
    lr=0.0001
)

# Função para treinar e validar o modelo
def treinar_e_validar(modelo, classifier, metrica_erro, otimizador_sgd, epocas=25, iniciar_epoca=0, melhor_acuracia=0.0):
    scaler = torch.cuda.amp.GradScaler()  # Inicializar GradScaler para Mixed Precision
    historico = []

    for epoca in range(iniciar_epoca, epocas):
        inicio_epoca = time.time()
        print(f"\nÉpoca {epoca + 1}/{epocas}")

        # Modo de treinamento
        modelo.train()
        classifier.train()
        erro_treino = 0.0
        acuracia_treino = 0.0

        for entradas, labels in tqdm(data_loader_treino, desc="Treinando"):
            entradas, labels = entradas.to(device), labels.to(device)
            otimizador_sgd.zero_grad()

            # Forward pass
            with torch.cuda.amp.autocast():  # Mixed Precision
                features = modelo(entradas)  # Extrair features do ViT
                saidas = classifier(features)  # Passar pelo classificador
                erro = metrica_erro(saidas, labels)  # Calcular perda

            # Backward pass
            scaler.scale(erro).backward()
            scaler.step(otimizador_sgd)
            scaler.update()

            erro_treino += erro.item() * entradas.size(0)
            _, preds = torch.max(saidas, 1)
            acuracia_treino += torch.sum(preds == labels.data)

        # Modo de avaliação
        modelo.eval()
        classifier.eval()
        erro_validacao = 0.0
        acuracia_validacao = 0.0

        with torch.no_grad():
            for entradas, labels in tqdm(data_loader_validacao, desc="Validando"):
                entradas, labels = entradas.to(device), labels.to(device)
                with torch.cuda.amp.autocast():
                    features = modelo(entradas)
                    saidas = classifier(features)
                    erro = metrica_erro(saidas, labels)

                erro_validacao += erro.item() * entradas.size(0)
                _, preds = torch.max(saidas, 1)
                acuracia_validacao += torch.sum(preds == labels.data)

        # Calcular métricas
        erro_medio_treino = erro_treino / len(data['treino'])
        acuracia_medio_treino = acuracia_treino.double() / len(data['treino'])
        erro_medio_validacao = erro_validacao / len(data['validacao'])
        acuracia_medio_validacao = acuracia_validacao.double() / len(data['validacao'])

        historico.append([erro_medio_treino, erro_medio_validacao, acuracia_medio_treino, acuracia_medio_validacao])

        print(f"Treino - Erro: {erro_medio_treino:.4f}, Acurácia: {acuracia_medio_treino:.4f}")
        print(f"Validação - Erro: {erro_medio_validacao:.4f}, Acurácia: {acuracia_medio_validacao:.4f}")

        # Salvar checkpoints
        checkpoint_path = os.path.join(checkpoint_dir, f'Vit_checkpoint_epoca_{epoca + 1}.pth')
        salvar_checkpoint(modelo, classifier, otimizador_sgd, epoca + 1, melhor_acuracia, checkpoint_path)

        # Atualizar o melhor modelo
        if acuracia_medio_validacao > melhor_acuracia:
            melhor_acuracia = acuracia_medio_validacao
            torch.save(modelo.state_dict(), 'melhor_modelo_vit.pth')
            torch.save(classifier.state_dict(), 'melhor_classifier.pth')
            print("Melhor modelo salvo!")

    return historico

# Treinar o modelo
historico = treinar_e_validar(vit_model, classifier, funcao_erro, otimizador, numero_de_epocas)

Usando o dispositivo: cuda:0


ImportError: attempted relative import with no known parent package